In [4]:
import pandas as pd
import nltk
nltk.download('punkt')

from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

from datasets import load_dataset, Dataset

from transformers import  BartTokenizer,BartForConditionalGeneration, TrainingArguments, Trainer






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/news.tsv"
#DATA_PATH = "D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/Data/news.tsv"
df_full = pd.read_csv(DATA_PATH,sep="\t")
df_full.head()

,News ID,Category,Topic,Headline,News body,Title entity,Entity content
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...","{""Atlanta United's"": 'Atlanta United FC'}","{'Atlanta United FC': {'type': 'item', 'id': '..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,"{'DC': 'Washington, D.C.'}","{'Washington, D.C.': {'type': 'item', 'id': 'Q..."
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,{},{}
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,{},{}
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",{},{}


In [7]:
df_full.count()

,0
News ID,113762
Category,113762
Topic,113762
Headline,113762
News body,113704
Title entity,113762
Entity content,113762


In [ ]:
#Randomly sample 10,000 rows
import pandas as pd

# Load data
df = pd.read_csv("/content/drive/MyDrive/news.tsv", sep="\t")

# Randomly sample 10,000 rows
df = df.sample(n=10000, random_state=42).reset_index(drop=True)


In [10]:
df.count()

,0
News ID,10000
Category,10000
Topic,10000
Headline,10000
News body,9996
Title entity,10000
Entity content,10000


In [ ]:
df.to_csv("/content/drive/MyDrive/Summarization/Cleaned_10000_recordstput.csv", index=False)


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Summarization/Cleaned_10000_recordstput.csv"

df = pd.read_csv(DATA_PATH)
df.head()

,News ID,Category,Topic,Headline,News body,Title entity,Entity content
0,N107100,entertainment,entertainment-celebrity,Stars who came out,Find out how these celebs revealed their sexua...,{},{}
1,N104494,foodanddrink,recipes,19 Ice Cream Pies You'll Want to Make All Summ...,Everyone will want a second slice! Cherry and ...,{},{}
2,N37185,finance,financenews,Mixed-used development will bring variety to d...,Seven of 26 expected vendors have already sign...,{},{}
3,N54647,news,newspolitics,Paul Manafort Seemed Headed to Rikers. Then th...,[What you need to know to start the day: Get N...,{'Justice Department': 'United States Departme...,{'United States Department of Justice': {'type...
4,N112983,sports,mma,Sean Shelby's Shoes: What's next for Junior Do...,(ALSO SEE: Sean Shelby's Shoes: What's next fo...,"{'Junior Dos Santos': 'Junior dos Santos', 'UF...","{'Junior dos Santos': {'type': 'item', 'id': '..."


In [14]:
df.columns

Index(['News ID', 'Category', 'Topic', 'Headline', 'News body', 'Title entity',
       'Entity content'],
      dtype='object')

In [15]:
df['Category'].value_counts()

,count
Category,
sports,2802
news,2339
finance,952
lifestyle,607
foodanddrink,492
autos,483
travel,471
video,391
tv,355


In [16]:
CANDIDATES = ["News body", "body", "text", "content", "article", "news"]
text_col = None
for c in CANDIDATES:
    if c in df.columns:
        text_col = c
        break

if text_col is None:
    # Fallback: choose the column with the longest average string length
    str_cols = [c for c in df.columns if df[c].dtype == "object"]
    if not str_cols:
        raise ValueError("❌ No string/text columns found in PENS dataset.")
    avg_lengths = {c: df[c].astype(str).str.len().mean() for c in str_cols}
    text_col = max(avg_lengths, key=avg_lengths.get)

print(f"✅ Using column '{text_col}' as article text.")

✅ Using column 'News body' as article text.


In [17]:
# Standardize text column name to 'text'
df["text"] = df[text_col].astype(str)

In [18]:
df.head()

,News ID,Category,Topic,Headline,News body,Title entity,Entity content,text
0,N107100,entertainment,entertainment-celebrity,Stars who came out,Find out how these celebs revealed their sexua...,{},{},Find out how these celebs revealed their sexua...
1,N104494,foodanddrink,recipes,19 Ice Cream Pies You'll Want to Make All Summ...,Everyone will want a second slice! Cherry and ...,{},{},Everyone will want a second slice! Cherry and ...
2,N37185,finance,financenews,Mixed-used development will bring variety to d...,Seven of 26 expected vendors have already sign...,{},{},Seven of 26 expected vendors have already sign...
3,N54647,news,newspolitics,Paul Manafort Seemed Headed to Rikers. Then th...,[What you need to know to start the day: Get N...,{'Justice Department': 'United States Departme...,{'United States Department of Justice': {'type...,[What you need to know to start the day: Get N...
4,N112983,sports,mma,Sean Shelby's Shoes: What's next for Junior Do...,(ALSO SEE: Sean Shelby's Shoes: What's next fo...,"{'Junior Dos Santos': 'Junior dos Santos', 'UF...","{'Junior dos Santos': {'type': 'item', 'id': '...",(ALSO SEE: Sean Shelby's Shoes: What's next fo...


In [19]:
# ==========================================================
# 5. Clean Text for Summarization
# ==========================================================
import re
def clean_text(t: str) -> str:
    t = str(t)
    # Remove email addresses
    t = re.sub(r"\S+@\S+", " ", t)
    # Remove URLs
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    # Remove boilerplate/news artifacts
    t = re.sub(r"RELATED:.*", " ", t)
    t = re.sub(r"Contact .*", " ", t)
    t = re.sub(r"Follow .*", " ", t)
    t = re.sub(r"Credit:.*", " ", t)
    t = re.sub(r"This article originally appeared.*", " ", t)
    # Collapse multiple spaces/newlines
    t = re.sub(r"\s+", " ", t)
    return t.strip()

In [20]:
print("🔹 Cleaning text...")
df["text"] = df["text"].fillna("").astype(str).apply(clean_text)

🔹 Cleaning text...


In [21]:
df['text'][9999]

"A young woman is fighting for her life after authorities said she was struck by an impaired driver near the University of Central Florida campus, according to authorities. Yousuf Hasan, 25, appeared before a judge Sunday morning on several charges including felony DUI, and leaving the scene of an accident. The crash happened around 12:30 a.m. Saturday at Napiers Circle and Pasteur Drive, just off North Alafaya Trail. Authorities said Hasan was visibly impaired and very talkative. The victim's father told a judge during Hasan's first appearance that his daughter, a senior at UCF, likely won't survive. READ MORE: UCF senior in coma after being struck by impaired driver, father says CHECK OUT WESH: Stay in the know with the latest Orlando news, weather and sports. Get the top stories and all the scores from the team at WESH."

In [22]:
# Drop empty or too short texts
df = df[df["text"].str.len() > 80].reset_index(drop=True)
print("✅ Rows after basic cleaning:", len(df))

✅ Rows after basic cleaning: 9616


In [23]:
df['Category'].value_counts()

,count
Category,
sports,2723
news,2277
finance,938
lifestyle,580
foodanddrink,467
autos,463
travel,455
video,338
tv,333


In [24]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [25]:
# ==========================================================
# 6. Generate Extractive Summaries (TextRank)
# ==========================================================
def auto_summary(text: str, num_sentences: int = 3) -> str:
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    textrank_summarizer = TextRankSummarizer()
    sentences = textrank_summarizer(parser.document, num_sentences)
    return " ".join(str(s) for s in sentences)

print("🔹 Generating pseudo-reference summaries with TextRank...")
df["summary"] = df["text"].apply(lambda x: auto_summary(x, num_sentences=4))

🔹 Generating pseudo-reference summaries with TextRank...


In [26]:
df['summary'].head(10)

,summary
0,"""I heard so many things from within the queer ..."
1,"Judy Wilson, Sun City West, Arizona Get Recipe..."
2,Bottleworks Will Be A Mixed Use Neighborhood I...
3,The intervention of Mr. Rosen was just the lat...
4,Junior Dos Santos ' winning streak came to a h...
5,They only made a few additions in free agency ...
6,One cup contains just 56 calories and is packe...
7,French Grand Prix officials have completely ch...
8,Catch Jasmine Roth on season 2 of HGTV's 'Hidd...
9,After Jeff Lewis compared his ex Gage Edward's...


In [27]:
len(df['summary'])

9616

In [28]:
df['summary'][9467]

'Adding a high-stakes immigration case to its election-year agenda, the Supreme Court said Friday it will decide whether President Donald Trump can terminate an Obama-era program shielding young migrants from deportation. (June 28)'

In [29]:
# Filter out any bad summaries
df = df[df["summary"].str.len() > 40].reset_index(drop=True)
print("✅ Rows after removing bad summaries:", len(df))

✅ Rows after removing bad summaries: 9616


In [ ]:
# Save cleaned dataset with summaries
CLEAN_CSV_PATH = '/content/drive/MyDrive/Summarization/pens_clean_with_summaries.csv'
df[["text", "summary"]].to_csv(CLEAN_CSV_PATH, index=False)
print("✅ Cleaned dataset with summaries saved to:", CLEAN_CSV_PATH)

✅ Cleaned dataset with summaries saved to: /content/drive/MyDrive/DSIPYNB/Summarization/pens_clean_with_summaries.csv


In [ ]:
# ==========================================================
# 1. Imports
# ==========================================================
import torch
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# ==========================================================
# 2. Set device
# ==========================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
if device == "cuda":
    print("GPU detected:", torch.cuda.get_device_name(0))

# ==========================================================
# 3. Load your dataset (replace 'df' with your DataFrame)
# ==========================================================
# Assuming your DataFrame 'df' has columns 'text' and 'summary'
raw_dataset = Dataset.from_pandas(df[["text", "summary"]])
raw_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
print(raw_dataset)

# ==========================================================
# 4. Load Pretrained BART
# ==========================================================
MODEL_NAME = "facebook/bart-large-cnn"
print("Loading tokenizer & model:", MODEL_NAME)
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)
model = model.to(device)
print("Model loaded on", device)

# ==========================================================
# 5. Tokenization Function
# ==========================================================
def preprocess(batch):
    articles = [str(x) for x in batch["text"]]
    summaries = [str(x) for x in batch["summary"]]

    model_inputs = tokenizer(
        articles,
        max_length=512,
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        summaries,
        max_length=160,
        padding="max_length",
        truncation=True,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("🔹 Tokenizing train and test splits...")
tokenized_dataset = raw_dataset.map(
    preprocess,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
)
print(tokenized_dataset)

# ==========================================================
# 6. Training Arguments
# ==========================================================
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Summarization/checkpoints",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # simulate larger batch
    learning_rate=2e-5,
    fp16=True,  # mixed precision for GPU
    save_steps=1000,
    save_total_limit=2,
    logging_steps=500,
    eval_strategy="epoch",
    report_to="none",  # disables Weights & Biases
)

# ==========================================================
# 7. Trainer Setup
# ==========================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# ==========================================================
# 8. Start Training
# ==========================================================
print("🔹 Starting training on", device)
trainer.train()
print("Training finished.")


Using device: cuda
GPU detected: Tesla T4
DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 8654
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 962
    })
})
Loading tokenizer & model: facebook/bart-large-cnn


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8654
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 962
    })
})
🔹 Starting training on cuda


Epoch,Training Loss,Validation Loss
1,4.792790,0.526824
2,3.579797,0.518070


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Training finished.


In [ ]:



# ==========================================================
# 12. Save Fine-tuned Model for Streamlit Use (Corrected)
# ==========================================================
MODEL_SAVE_PATH="/content/drive/MyDrive/Summarization/bart_Summary_finetuned"

# Save model weights
model.save_pretrained(MODEL_SAVE_PATH)

# Save tokenizer
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print("✅ Fine-tuned model saved to:", MODEL_SAVE_PATH)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Fine-tuned model saved to: /content/drive/MyDrive/DSIPYNB/Summarization/bart_Summary_finetuned


In [34]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [35]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f7da8e6cb65e1ba80d8455c3086cf30cb4def9d2ba7d542f8af22f4f27b28ac7
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [38]:
# ==========================================================
# 13. ROUGE Evaluation on Test Set (SAFE VERSION)
# ==========================================================

print("🔹 Evaluating with ROUGE on test set...")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_PATH)
model.to("cuda")
model.eval()



# ----------------------------------------------------------
# SAFE summarization wrapper to avoid CUDA crashes
# ----------------------------------------------------------
def safe_summarize(article):

    inputs = tokenizer(
        article,
        truncation=True,
        padding="longest",
        max_length=1024,
        return_tensors="pt"
    ).to("cuda")

    input_len = inputs["input_ids"].shape[1]

    max_len = min(180, max(32, int(input_len * 0.75)))
    min_len = min(max_len - 5, max(15, int(max_len * 0.5)))

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=5,
            no_repeat_ngram_size=3,
            repetition_penalty=2.0,
            early_stopping=True
        )

    summary = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )

    if not summary.strip().endswith("."):
        summary = summary.rsplit(".", 1)[0] + "."

    return summary




# ----------------------------------------------------------
# Generate summaries on test samples
# ----------------------------------------------------------
predictions = []
references = []
texts = []

MAX_EVAL_SAMPLES = min(300, len(raw_dataset["test"]))   # speed-safe

for i in range(MAX_EVAL_SAMPLES):

    sample = raw_dataset["test"][i]

    article = sample["text"]
    ref_sum = sample["summary"]

    pred_sum = safe_summarize(article)

    predictions.append(pred_sum)
    references.append(ref_sum)
    texts.append(article)


# ----------------------------------------------------------
# Compute ROUGE
# ----------------------------------------------------------
rouge_scores = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True,
)

print("✅ ROUGE scores:")
print(rouge_scores)





🔹 Evaluating with ROUGE on test set...


Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

✅ ROUGE scores:
{'rouge1': np.float64(0.6894419226438848), 'rouge2': np.float64(0.6166236829098273), 'rougeL': np.float64(0.6332797129160187), 'rougeLsum': np.float64(0.6325237292508361)}


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# SAVE RESULTS TO CSV
# ---------------------------------------------------------
OUTPUT_CSV = "/content/drive/MyDrive/Summarization/rouge_eval_results.csv"  # change if needed
result = rouge_scores  # just for clarity

new_results = pd.DataFrame([
    {
        "Model": "BART-Large-CNN",
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
        "rougeLsum": result["rougeLsum"],
        "Average Score": np.mean([
            result["rouge1"],
            result["rouge2"],
            result["rougeL"],
        ])
    }
])

# Round to 4 decimals
new_results = new_results.round(4)

# Append if CSV exists, else create
try:
    old_df = pd.read_csv(OUTPUT_CSV)
    final_df = pd.concat([old_df, new_results], ignore_index=True)
except FileNotFoundError:
    final_df = new_results

final_df.to_csv(OUTPUT_CSV, index=False)

print("\n✅ Evaluation Completed.")
print("Saved to:", OUTPUT_CSV)



✅ Evaluation Completed.
Saved to: /content/drive/MyDrive/DSIPYNB/Summarization/rouge_eval_results.csv


In [41]:
import pandas as pd

final_comparison_df = pd.read_csv(OUTPUT_CSV)

best_overall_Average = final_comparison_df.loc[
    final_comparison_df["Average Score"].idxmax()
]


In [42]:
best_overall_Average

,3
Model,BART-Large-CNN
rouge1,0.6894
rouge2,0.6166
rougeL,0.6333
rougeLsum,0.6325
Average Score,0.6464


In [43]:
best_overall_rougeL = final_comparison_df.loc[
    final_comparison_df["rougeL"].idxmax()
]


In [44]:
best_overall_rougeL

,3
Model,BART-Large-CNN
rouge1,0.6894
rouge2,0.6166
rougeL,0.6333
rougeLsum,0.6325
Average Score,0.6464


In [46]:
model_type_map = {
    "TextRank": "Extractive",
    "TF-IDF": "Extractive",
    "Seq2Seq LSTM": "Abstractive",
    "BART-Large-CNN": "Transformer"
}

In [48]:
final_comparison_df["Type"] = final_comparison_df["Model"].map(model_type_map)

In [50]:
final_comparison_df.to_csv(OUTPUT_CSV, index=False)


In [53]:


final_comparison_df = pd.read_csv(OUTPUT_CSV)

best_overall_Average = final_comparison_df.loc[
    final_comparison_df["Average Score"].idxmax()
]


In [54]:
print("\n==================== 🏆 FINAL SELECTED MODEL 🏆 ====================")
print("Model Name      :", best_overall_Average["Model"])
print("ROUGE-1         :", round(best_overall_Average["rouge1"], 4))
print("ROUGE-2         :", round(best_overall_Average["rouge2"], 4))
print("ROUGE-L         :", round(best_overall_Average["rougeL"], 4))
print("ROUGE-Lsum      :", round(best_overall_Average["rougeLsum"], 4))
print("Average ROUGE   :", round(best_overall_Average["Average Score"], 4))
print("Model Type      :", best_overall_Average["Type"])


==================== 🏆 FINAL SELECTED MODEL 🏆 ====================
Model Name      : BART-Large-CNN
ROUGE-1         : 0.6894
ROUGE-2         : 0.6166
ROUGE-L         : 0.6333
ROUGE-Lsum      : 0.6325
Average ROUGE   : 0.6464
Model Type      : Transformer
